In [1]:
import PISM
import os
import numpy as np
from PISM.testing import *

import numpy as np
import pylab as plt

config = PISM.Context().config

# reduce the grid size to speed this up
config.set_number("grid.Mx", 3)
config.set_number("grid.My", 5)
config.set_number("grid.Mz", 5)

seconds_per_year = 365 * 86400
# ensure that this is the correct year length
config.set_string("time.calendar", "365_day")
config.set_flag("ocean.th.clip_salinity", False)

# silence models' initialization messages
PISM.Context().log.set_threshold(1)

ice_density = config.get_number("constants.ice.density")
sea_water_density = config.get_number("constants.sea_water.density")

ModuleNotFoundError: No module named 'PISM'

In [ ]:
def melting_point_temperature(ice_thickness, salinity):
        a = [-0.0575, 0.0901, -7.61e-4]
        return a[0] * salinity + a[1] + a[2] * ice_thickness

In [ ]:
h = np.arange(800, 1000, 100)
t = np.arange(4, 6, 1)
H, T = np.meshgrid(h, t)
nx, ny = T.shape

M = np.zeros_like(H)

gamma_T = 1.00e-4
gamma_S = 5.05e-7


for ice_thickness in h:
    for temperature in t:
        depth = ice_density / sea_water_density * ice_thickness
        salinity = 35.0
        melange_back_pressure = 0.0
        potential_temperature = temperature - melting_point_temperature(ice_thickness, salinity) + 273.13

        grid = shallow_grid()
        geometry = PISM.Geometry(grid)

        geometry.ice_thickness.set(depth)

        filename = "ocean_given_th_input.nc"

        PISM.util.prepare_output(filename)

        Th = PISM.IceModelVec2S(grid, "theta_ocean", PISM.WITHOUT_GHOSTS)
        Th.set_attrs("climate", "potential temperature", "Kelvin", "Kelvin", "", 0)
        Th.set(potential_temperature)
        Th.write(filename)

        S = PISM.IceModelVec2S(grid, "salinity_ocean", PISM.WITHOUT_GHOSTS)
        S.set_attrs("climate", "ocean salinity", "g/kg", "g/kg", "", 0)
        S.set(salinity)
        S.write(filename)

        config.set_string("ocean.th.file", filename)
        config.set_number("ocean.th.gamma_T", gamma_T)
        config.set_number("ocean.th.gamma_S", gamma_S)

        model = PISM.OceanGivenTH(grid)
        model.init(geometry)
        model.update(geometry, 0, 1)
        m = model.shelf_base_mass_flux().numpy()[0, 0]
        m = m * seconds_per_year / ice_density
        print(f"H={ice_thickness} m, TH={potential_temperature-273.13:.2f} C, S={salinity} g/kg, m={m:.2f} m/yr")


In [ ]:
        gamma_T = 1.3e-4
        gamma_S = 5.05e-7

        ice_thickness = 650
        depth = ice_density / sea_water_density * ice_thickness
        salinity = 34.0
        melange_back_pressure = 0.0
        potential_temperature = 4.2 + 273.13
        # temperature = 1.7
        # potential_temperature = temperature - melting_point_temperature(ice_thickness, salinity) + 273.13

        grid = shallow_grid()
        geometry = PISM.Geometry(grid)

        geometry.ice_thickness.set(depth)

        filename = "ocean_given_th_input.nc"

        PISM.util.prepare_output(filename)

        Th = PISM.IceModelVec2S(grid, "theta_ocean", PISM.WITHOUT_GHOSTS)
        Th.set_attrs("climate", "potential temperature", "Kelvin", "Kelvin", "", 0)
        Th.set(potential_temperature)
        Th.write(filename)

        S = PISM.IceModelVec2S(grid, "salinity_ocean", PISM.WITHOUT_GHOSTS)
        S.set_attrs("climate", "ocean salinity", "g/kg", "g/kg", "", 0)
        S.set(salinity)
        S.write(filename)

        config.set_string("ocean.th.file", filename)
        config.set_number("ocean.th.gamma_T", gamma_T)
        config.set_number("ocean.th.gamma_S", gamma_S)


        model = PISM.OceanGivenTH(grid)
        model.init(geometry)
        model.update(geometry, 0, 1)
        m = model.shelf_base_mass_flux().numpy()[0, 0]
        m = m * seconds_per_year / ice_density
        print(f"H={ice_thickness} m, TH={potential_temperature-273.13:.2f} C, S={salinity} g/kg, m={m:.2f} m/yr")



In [26]:
run create_jib_ocean_forcing.py --accelerator cpu

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/andy/miniforge3/envs/pism-emulator/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name       | Type                       | Params | Mode 
------------------------------------------------------------------
0 | likelihood | GaussianLikelihood         | 1      | train
1 | model      | MultitaskGPModel           | 6      | train
2 | mll        | ExactMarginalLogLikelihood | 6      | train
------------------------------------------------------------------
6         Trainable params
0         Non-trainable params
6         Total params
0.000     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Training Temperature [Celsius]
Setting up MultitaksGPModel with RBFKernel(
  (raw_lengthscale_constraint): Positive()
)


/Users/andy/miniforge3/envs/pism-emulator/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/andy/miniforge3/envs/pism-emulator/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Temperature [Celsius]: calculating sample mean
Temperature [Celsius]: sampling from distribution


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type                       | Params | Mode 
------------------------------------------------------------------
0 | likelihood | GaussianLikelihood         | 1      | train
1 | model      | MultitaskGPModel           | 6      | train
2 | mll        | ExactMarginalLogLikelihood | 6      | train
------------------------------------------------------------------
6         Trainable params
0         Non-trainable params
6         Total params
0.000     Total estimated model params size (MB)
7         Modules in train mode
3         Modules in eval mode


Training Salinity [g/kg]
Setting up MultitaksGPModel with RBFKernel(
  (raw_lengthscale_constraint): Positive()
)


Training: |                                                                                                   …

Salinity [g/kg]: calculating sample mean
Salinity [g/kg]: sampling from distribution
Processing Bay
  Writing jib_ocean_forcing_id_bay_0_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_1_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_2_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_3_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_4_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_5_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_6_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_7_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_8_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_9_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_ctrl_1980_2020.nc
  Writing jib_ocean_forcing_id_bay_tm_1985_1995.nc
Processing Fjord
  Writing jib_ocean_forcing_id_fjord_0_1980_2020.nc
  Writing jib_ocean_forcing_id_fjord_1_1980_2020.nc
  Writing jib_ocean_forcing_id_fjord_2_1980_2020.nc
  Writing jib_ocean_forcing_id_fjord_3_1980_2020.nc
  Writing jib_ocean_forcing_id_fjord_4_1980_2020.nc
  Writing jib_ocean

In [22]:
ginr

,Temperature [Celsius],Salinity [g/kg]
Date,,
1991-09-13,1.156297,34.014921
1991-09-14,1.038524,33.965140
1991-09-17,1.059300,33.982285
1991-09-18,1.181707,34.027700
1991-09-22,1.158413,33.986539
1994-08-22,1.026046,34.002623
1994-08-23,1.155961,34.053970
1994-08-24,1.418294,34.152282
1994-08-25,1.143185,34.008696


In [14]:
resolution = 0.01
np.arange(-54, -31 + resolution , resolution)

array([-54.  , -53.99, -53.98, ..., -31.04, -31.03, -31.02], shape=(2299,))

In [ ]:
ices = pd.read_csv("disko_bay_ices.csv")



In [ ]:
df = ices[(ices["PRES [db]"] >= 225) & (ices["PRES [db]"] <= 275)][["yyyy-mm-ddThh:mm", "TEMP [deg C]"]]

In [ ]:
df.rename(columns={"yyyy-mm-ddThh:mm": "time", "TEMP [deg C]": "temperature"}).reset_index(drop=True)

In [17]:
resolution = 0.01
lon = np.arange(-50.0, -20.0 + resolution , resolution)
lat_1 = np.zeros_like(lon) + 67
lat_2 = np.zeros_like(lon) + 72
lat_3 = np.zeros_like(lon) + 77
[(x, y) for x, y in zip(lon, lat_1)]

[(np.float64(-50.0), np.float64(67.0)),
 (np.float64(-49.99), np.float64(67.0)),
 (np.float64(-49.980000000000004), np.float64(67.0)),
 (np.float64(-49.970000000000006), np.float64(67.0)),
 (np.float64(-49.96000000000001), np.float64(67.0)),
 (np.float64(-49.95000000000001), np.float64(67.0)),
 (np.float64(-49.94000000000001), np.float64(67.0)),
 (np.float64(-49.930000000000014), np.float64(67.0)),
 (np.float64(-49.920000000000016), np.float64(67.0)),
 (np.float64(-49.91000000000002), np.float64(67.0)),
 (np.float64(-49.90000000000002), np.float64(67.0)),
 (np.float64(-49.89000000000002), np.float64(67.0)),
 (np.float64(-49.880000000000024), np.float64(67.0)),
 (np.float64(-49.870000000000026), np.float64(67.0)),
 (np.float64(-49.86000000000003), np.float64(67.0)),
 (np.float64(-49.85000000000003), np.float64(67.0)),
 (np.float64(-49.84000000000003), np.float64(67.0)),
 (np.float64(-49.830000000000034), np.float64(67.0)),
 (np.float64(-49.820000000000036), np.float64(67.0)),
 (np.float

In [ ]:
import scipy.stats as stats
import math

mu = 0
variance = 0.2
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.show()

In [ ]:
pm.Normal?

In [ ]:
!open disko-bay-temps_*.pdf

In [ ]:
    import matplotlib.pyplot as plt
    import numpy as np
    import scipy.stats as st
    plt.style.use('seaborn-darkgrid')
    x = np.linspace(-5, 5, 1000)
    mus = [0.]
    sigmas = [0.2]
    for mu, sigma in zip(mus, sigmas):
        pdf = st.norm.pdf(x, mu, sigma)
        plt.plot(x, pdf, label=r'$\mu$ = {}, $\sigma$ = {}'.format(mu, sigma))
    plt.xlabel('x', fontsize=12)
    plt.ylabel('f(x)', fontsize=12)
    plt.legend(loc=1)
    plt.show()
    


In [ ]:
def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = datetime(year=year, month=1, day=1)
    startOfNextYear = datetime(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

In [ ]:
[toYearFraction(d) for d in time_df]

In [ ]:
time_df

In [ ]:
fig, ax = plt.subplots(3, 1, sharex="col", sharey="row", figsize=[4.75, 5.25])
fig.subplots_adjust(hspace=0.05, wspace=0.30)


# plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax[0], pred_samples_db["f_pred"], X_new, palette="Reds");
plot_gp_dist(ax[1], pred_samples_mo["f_pred"], X_new, palette="Blues");
plot_gp_dist(ax[2], pred_samples_all["f_pred"], X_new, palette="Greens");

# plot the data and the true latent function
# plt.plot(X, f_true, "dodgerblue", lw=3, label="True f");
ax[0].plot(X_db, y_db, 'or', ms=3, alpha=0.5, label="Observed data Khazendar");
ax[1].plot(X_mo, y_mo, 'ob', ms=3, alpha=0.5, label="Observed data Motyka");
ax[2].plot(X_db, y_db, 'or', ms=3, alpha=0.5, label="Observed data Khazendar");
ax[2].plot(X_mo, y_mo, 'ob', ms=3, alpha=0.5, label="Observed data Motyka");


# axis labels and title
ax[2].set_xlabel("Time")
ax[0].set_ylabel("Temperature (Celsius)")
ax[1].set_ylabel("Temperature (Celsius)")
ax[2].set_ylabel("Temperature (Celsius)")

ax[0].set_ylim(0, 5)
ax[1].set_ylim(0, 5)
ax[2].set_ylim(0, 5)

In [ ]:
fig.savefig("disko-bay-mcmc.pdf")

How many samples should I draw? Is it the number of samples to use for the high-fidelity model forcing?

In [ ]:
!open disko-bay-mcmc.pdf

In [ ]:
fig.add_subplot?

In [ ]:
pm.Normal?

In [ ]:
# What is the "nu" for??
with pm.Model() as gp:
    ρ_db = pm.Gamma('ρ_db', 3, 2)
    η_db = pm.Gamma('η_db', 2, 2)
    K_db = pm.gp.cov.Matern32(1, ρ_db)
    
    ρ_mo = pm.Gamma('ρ_mo', 3, 2)
    η_mo = pm.Gamma('η_mo', 3, 2)
    K_mo = η_mo * pm.gp.cov.Matern32(1, ρ_mo)

    # Noise
    σ = pm.Normal('σ', 0.2)
    
    gp_db = pm.gp.Marginal(cov_func=K_db)
    gp_mo = pm.gp.Marginal(cov_func=K_mo)

    gp = gp_db + gp_mo
    
    y_ = gp.marginal_likelihood("y", X=X_all, y=y_all, noise=σ)
        
    # conditional distributions of f1 and f2
    f1_star = gp_db.conditional("f1_star", X_new,
                              given={"X": X_db, "y": y_db, "noise": σ, "gp": gp})
    f2_star = gp_mo.conditional("f2_star", X_new,
                              given={"X": X_mo, "y": y_mo, "noise": σ, "gp": gp})

    # conditional of f1 + f2, `given` not required
    f_star = gp.conditional("f_star", X_new)
    
    mp = pm.find_MAP()
    pred_samples = pm.sample_posterior_predictive([mp], vars=[f_star], samples=2000)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex="col", sharey="row", figsize=[4.75, 5.25])
fig.subplots_adjust(hspace=0.25, wspace=0.35)


# plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax[0], pred_samples_all["f_pred"], X_new, palette="Greens");
plot_gp_dist(ax[1], pred_samples["f_star"], X_new, palette="Blues");


# plot the data and the true latent function
# plt.plot(X, f_true, "dodgerblue", lw=3, label="True f");
ax[0].plot(X_db, y_db, 'or', ms=3, alpha=0.5, label="Observed data Khazendar");
ax[0].plot(X_mo, y_mo, 'ob', ms=3, alpha=0.5, label="Observed data Motyka");
ax[1].plot(X_db, y_db, 'or', ms=3, alpha=0.5, label="Observed data Khazendar");
ax[1].plot(X_mo, y_mo, 'ob', ms=3, alpha=0.5, label="Observed data Motyka");



# axis labels and title
ax[1].set_xlabel("Time")
ax[0].set_ylabel("Temperature (Celsius)")
ax[1].set_ylabel("Temperature (Celsius)")

ax[0].set_ylim(0, 5)
ax[1].set_ylim(0, 5)


In [ ]:
run create_disko_bay_timeseries.py

In [ ]:
        theta_ocean = temperate - melting_point_temperature(depth, salinity)


In [ ]:
NC?

In [ ]:
nx = 3
ny = 4
nt = len(theta_ocean)
np.repeat(theta_ocean, nx*ny).reshape(nt, ny, nx).shape

In [ ]:
time_interval_since_refdate

In [ ]:
len(bnds_interval_since_refdate)

In [ ]:
    start_date = datetime(1980, 1, 1)
    end_date = datetime(2020, 1, 1)
    end_date_yearly = datetime(2020, 1, 1)

    # create list with dates from start_date until end_date with
    # periodicity prule.
    bnds_datelist = list(rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date_yearly))
    bnds_datelist_yearly = list(rrule.rrule(rrule.YEARLY, dtstart=start_date, until=end_date_yearly))

    # calculate the days since refdate, including refdate, with time being the
    bnds_interval_since_refdate = cdftime_days.date2num(bnds_datelist)
    bnds_interval_since_refdate_yearly = cdftime_days.date2num(bnds_datelist_yearly)
    time_interval_since_refdate = bnds_interval_since_refdate[0:-1] + np.diff(bnds_interval_since_refdate) / 2


In [ ]:
    start_date = datetime(1980, 1, 1)
    end_date = datetime(2020, 1, 1)
    end_date_yearly = datetime(2020, 1, 2)

    # create list with dates from start_date until end_date with
    # periodicity prule.
    bnds_datelist = list(rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date))
    bnds_datelist_yearly = list(rrule.rrule(rrule.YEARLY, dtstart=start_date, until=end_date_yearly))

    # calculate the days since refdate, including refdate, with time being the
    bnds_interval_since_refdate = cdftime_days.date2num(bnds_datelist)
    bnds_interval_since_refdate_yearly = cdftime_days.date2num(bnds_datelist_yearly)
    time_interval_since_refdate = bnds_interval_since_refdate[0:-1] + np.diff(bnds_interval_since_refdate) / 2

    time_dict = {
        "calendar": calendar,
        "units": units,
        "time": time_interval_since_refdate,
        "time_bnds": bnds_interval_since_refdate,
    }

    dpy = np.diff(bnds_interval_since_refdate_yearly)
    dpy = np.repeat(dpy, 12, axis=0)

    decimal_time = bnds_interval_since_refdate[0:-1] / dpy + 1980


In [ ]:
pm.gp.cov.

In [ ]:
len(bnds_interval_since_refdate[0:-1])

In [1]:
run prepare_omg.py

Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171023170447.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014164755.nc


/Volumes/zachariae/crios2pism/data_sets/ocean/prepare_omg.py:24: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  temperature = np.squeeze(nc.variables["temperature"][:]).reshape(-1, 1)
/Volumes/zachariae/crios2pism/data_sets/ocean/prepare_omg.py:25: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  lat = np.repeat(nc.geospatial_lat_min, n).reshape(-1, 1)
/Volumes/zachariae/crios2pism/data_sets/ocean/prepare_omg.py:26: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  lon = np.repeat(nc.geospatial_lon_min, n).reshape(-1, 1)
/Volumes/zachariae/crios2pism/data_sets/ocean/prepare_omg.py:27: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  time = np.repeat(nc.time_coverage_start, n).reshape(-1, 1)


Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014134531.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171019160601.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171016151617.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018172739.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171017161959.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018153909.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018140759.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014101700.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018113411.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018114821.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018164026.nc
Processing /Volumes/files/allDat

Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171019145057.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018144247.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014152026.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171019151620.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014141743.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171023153028.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014105649.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014094217.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171023161628.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018154526.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171016122834.nc
Processing /Volumes/files/allDat

Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171015101527.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171017161013.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171023165722.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018135128.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171023161928.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171014162838.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171015130605.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171018131029.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171015125030.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171019151952.nc
Processing /Volumes/files/allData/omg/L2/CTD/AXCTD/OMG_Ocean_AXCTD_L2_20171019130802.nc
Processing /Volumes/files/allDat

In [2]:
df

,date,lat,lon,depth,temperature
0,2017-10-23T17:04:47,68.61205,-53.8931,0.72,2.79
1,2017-10-23T17:04:47,68.61205,-53.8931,0.8304,2.79
2,2017-10-23T17:04:47,68.61205,-53.8931,0.9409,2.79
3,2017-10-23T17:04:47,68.61205,-53.8931,1.0513,2.79
4,2017-10-23T17:04:47,68.61205,-53.8931,1.1618,2.79
...,...,...,...,...,...
2014,2017-10-17T15:15:02,80.6111,-66.9596,221.6204,0.29
2015,2017-10-17T15:15:02,80.6111,-66.9596,221.7294,0.28
2016,2017-10-17T15:15:02,80.6111,-66.9596,221.8383,0.28
2017,2017-10-17T15:15:02,80.6111,-66.9596,221.9472,0.28


In [3]:
df.to_csv("test.csv")

In [ ]:
pd.DataFrame(data=np.hstack([time, depth, temperature]), columns=["date", "depth", "temperature"])

In [5]:
import geopandas

ImportError: dlopen(/opt/local/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fiona/ogrext.cpython-38-darwin.so, 2): Symbol not found: _GEOSArea
  Referenced from: /opt/local/lib/libspatialite.7.dylib
  Expected in: flat namespace
 in /opt/local/lib/libspatialite.7.dylib

In [4]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

NameError: name 'geopandas' is not defined

In [6]:
plt.cla?